In [ ]:
import numpy as np
import chess
import chess.svg
import math
import pygame
from pygame.locals import *
import random 

In [ ]:
# Pygame colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BOARD_SIZE = 600  # Increase the board size
SQUARE_SIZE = BOARD_SIZE // 8
INPUT_HEIGHT = 50  # Height of the input field


PIECE_VALUES = {
    'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9,
    'p': -1, 'n': -3, 'b': -3, 'r': -5, 'q': -9,
}

# Pawn structure evaluation function
def evaluate_pawn_structure(board):
    pawn_structure_value = 0
    
    # Count the number of isolated and doubled pawns for each side
    for color in [chess.WHITE, chess.BLACK]:
        isolated_pawns = 0
        doubled_pawns = 0
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is not None and piece.color == color and piece.piece_type == chess.PAWN:
                pawn_file = chess.square_file(square)
                pawn_rank = chess.square_rank(square)

                # Check for isolated pawns
                if (board.piece_at(chess.square(pawn_file - 1, pawn_rank - 1)) is None and
                    board.piece_at(chess.square(pawn_file, pawn_rank - 1)) is None and
                    board.piece_at(chess.square(pawn_file + 1, pawn_rank - 1)) is None):
                    isolated_pawns += 1

                # Check for doubled pawns
                if (board.piece_at(chess.square(pawn_file, pawn_rank - 1)) is not None):
                    doubled_pawns += 1

        # Assign values based on isolated and doubled pawns
        pawn_structure_value += (isolated_pawns * -10 + doubled_pawns * -5)

    return pawn_structure_value

# King safety evaluation function
def evaluate_king_safety(board):
    king_safety_value = 0

    # Find the squares of the kings for both sides
    white_king_square = board.king(chess.WHITE)
    black_king_square = board.king(chess.BLACK)

    # Assess the number of open files in front of each king
    white_king_file = chess.square_file(white_king_square)
    black_king_file = chess.square_file(black_king_square)

    for file in range(8):
        if file != white_king_file:
            king_safety_value += 5  # Award points for open files in front of the opponent's king

    for file in range(8):
        if file != black_king_file:
            king_safety_value -= 5  # Deduct points for open files in front of the AI's king

    return king_safety_value

# Key squares evaluation function (center squares and central files)
def evaluate_key_squares(board):
    key_squares_value = 0

    # Control of center squares
    center_squares = [chess.D4, chess.E4, chess.D5, chess.E5]
    for square in center_squares:
        piece = board.piece_at(square)
        if piece is not None:
            key_squares_value += PIECE_VALUES.get(piece.symbol(), 0)

    # Control of central files (d and e files)
    central_files = [3, 4]  # 3 represents file D, and 4 represents file E
    for file in central_files:
        for rank in range(8):
            square = chess.square(file, rank)
            piece = board.piece_at(square)
            if piece is not None:
                key_squares_value += PIECE_VALUES.get(piece.symbol(), 0)

    return key_squares_value



def evaluate_board(board):
    total_value = 0

    # Evaluate based on piece values and mobility
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            piece_value = PIECE_VALUES.get(piece.symbol(), 0)
            total_value += piece_value

            # Evaluate based on piece mobility
            if piece.color == board.turn:
                mobility = len(list(board.attacks(square)))
                total_value += mobility

    # Evaluate based on pawn structure, king safety, and control of key squares
    total_value += evaluate_pawn_structure(board)
    total_value += evaluate_king_safety(board)
    total_value += evaluate_key_squares(board)

    return total_value


def generate_moves(board):
    legal_moves = list(board.legal_moves)
    moves_with_captures_checks = []
    for move in legal_moves:
        if board.is_capture(move) or board.gives_check(move):
            moves_with_captures_checks.insert(0, move)
        else:
            moves_with_captures_checks.append(move)
    return [move.uci() for move in moves_with_captures_checks]

def generate_captures(board):
    # Generate capture moves
    capture_moves = []
    for move in board.legal_moves:
        if board.is_capture(move) or board.gives_check(move):
            capture_moves.append(move.uci())
    return capture_moves



def get_piece_type(piece):
    piece_type = piece.piece_type
    piece_names = {
        chess.PAWN: 'pawn',
        chess.ROOK: 'rook',
        chess.KNIGHT: 'knight',
        chess.BISHOP: 'bishop',
        chess.QUEEN: 'queen',
        chess.KING: 'king'
    }
    return piece_names.get(piece_type, 'unknown')

In [ ]:
'''class ChessNode:
    def __init__(self, board, parent=None, action_taken=None):
        self.board = board
        self.parent = parent
        self.action_taken = action_taken
        self.children = []
        self.visits = 0
        self.value_sum = 0

    def is_fully_expanded(self):
        if not self.children:
            return False

        explored_moves = set()
        for child in self.children:
            explored_moves.add(child.action_taken)

        legal_moves = list(self.board.legal_moves)
        return len(legal_moves) == len(explored_moves)

    def select(self):
        best_child = None
        best_ucb = -float('inf')

        for child in self.children:
                ucb = self.get_ucb(child)
                if ucb > best_ucb:
                    best_child = child
                    best_ucb = ucb

        return best_child


    def get_ucb(self, child):
        q_value = 1 - ((child.value_sum / child.visits) + 1) / 2
        return q_value + math.sqrt(1.41 * math.log(self.visits) / child.visits)

    def expand(self):
        legal_moves = list(self.board.legal_moves)
        untried_moves = [move for move in legal_moves if move not in [child.action_taken for child in self.children]]
        if untried_moves:
            action = random.choice(untried_moves)
            new_board = self.board.copy()
            new_board.push(action)
            new_child = ChessNode(new_board, parent=self, action_taken=action)
            self.children.append(new_child)
            return new_child
        return self 


    def simulate(self):
        board = self.board.copy()
        while not board.is_game_over():
            move = random.choice(list(board.legal_moves))
            board.push(move)
        return evaluate_board(board)
    
    def is_terminal(self):
        return self.board.is_game_over()

    def backpropagate(self, result):
        self.visits += 1
        self.value_sum += result
        if self.parent:
            self.parent.backpropagate(result)


class ChessMCTS:
    def __init__(self, max_iterations):
        self.max_iterations = max_iterations

    def search(self, board):
        root = ChessNode(board)

        for _ in range(self.max_iterations):
            node = root

            # Selection phase
            while not node.is_fully_expanded():
                if node.is_terminal():
                    break
                node = node.select()

            # Expansion phase
            if not node.is_terminal():
                node = node.expand()

            # Simulation phase
            result = node.simulate()

            # Backpropagation phase
            node.backpropagate(result)

        # Select the best move based on the most visited child
        best_child = max(root.children, key=lambda child: child.visits)
        best_move = best_child.action_taken
        return best_move


def ai_turn_mcts(board, max_iterations, player):
    mcts_agent = ChessMCTS(max_iterations=max_iterations)
    best_move = mcts_agent.search(board)
    board.push(best_move)
    evaluation = evaluate_board(board)  # Implement your board evaluation function
    print(f"Player {player}'s move (MCTS): {best_move.uci()}")
    print(f"Evaluation after Player {player}'s move: {evaluation}")
    return best_move


'''

In [ ]:
import chess
import numpy as np

class ChessGame:
    def __init__(self):
        self.board = chess.Board()
        self.action_size = len(list(self.board.legal_moves))

    def get_initial_state(self):
        return self.board.copy()

    def get_next_state(self, state, action, player):
        new_state = state.copy()
        new_state.push(action)
        return new_state

    def get_valid_moves(self, state):
        valid_moves = np.zeros(self.action_size)
        legal_moves = list(state.legal_moves)
        for move in legal_moves:
            move_index = self.move_to_index(move)
            valid_moves[move_index] = 1
        return valid_moves

    def check_win(self, state, action):
        return state.is_checkmate()

    def get_value_and_terminated(self, state, action):
        if self.check_win(state, action):
            return 1, True
        if state.is_stalemate() or state.is_insufficient_material() or state.is_seventyfive_moves() or state.is_variant_draw():
            return 0, True
        return 0, False

    def get_opponent(self, player):
        return -player

    def get_opponent_value(self, value):
        return -value

    def change_perspective(self, state, player):
        return state.copy()

    def move_to_index(self, move):
        return list(self.board.legal_moves).index(move)


In [ ]:
import chess
import chess.engine
import math
import numpy as np

class Node:
    def __init__(self, game, args, state, parent=None, action_taken=None):
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken

        self.children = []
        self.expandable_moves = self.game.get_valid_moves(state)

        self.visit_count = 0
        self.value_sum = 0

    def is_fully_expanded(self):
        return all(move == 0 for move in self.expandable_moves) and len(self.children) > 0

    def select(self):
        best_child = None
        best_ucb = -np.inf

        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb

        return best_child

    def get_ucb(self, child):
        q_value = 1 - ((child.value_sum / child.visit_count) + 1) / 2
        return q_value + self.args['C'] * math.sqrt(math.log(self.visit_count) / child.visit_count)

    def expand(self):
        action = np.random.choice(np.where(self.expandable_moves == 1)[0])
        self.expandable_moves[action] = 0

        child_state = self.state.copy()
        move = self.game.get_legal_moves(child_state)[action]
        child_state.push(move)

        child = Node(self.game, self.args, child_state, self, action)
        self.children.append(child)
        return child

    def simulate(self):
        value, is_terminal = self.game.get_value_and_terminated(self.state, self.action_taken)
        value = self.game.get_opponent_value(value)

        if is_terminal:
            return value

        rollout_state = self.state.copy()
        rollout_player = chess.WHITE
        while True:
            valid_moves = self.game.get_valid_moves(rollout_state)
            action = np.random.choice(np.where(valid_moves == 1)[0])
            move = self.game.get_legal_moves(rollout_state)[action]
            rollout_state.push(move)

            value, is_terminal = self.game.get_value_and_terminated(rollout_state, move)
            if is_terminal:
                if rollout_player == chess.BLACK:
                    value = self.game.get_opponent_value(value)
                return value

            rollout_player = self.game.get_opponent(rollout_player)

    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1

        value = self.game.get_opponent_value(value)
        if self.parent is not None:
            self.parent.backpropagate(value)


class ChessMCTS:
    def __init__(self, game, args):
        self.game = game
        self.args = args

    def search(self, state):
        root = Node(self.game, self.args, state)

        for _ in range(self.args['num_searches']):
            node = root

            while node.is_fully_expanded():
                node = node.select()

            value, is_terminal = self.game.get_value_and_terminated(node.state, node.action_taken)
            value = self.game.get_opponent_value(value)

            if not is_terminal:
                node = node.expand()
                value = node.simulate()

            node.backpropagate(value)

        action_probs = np.zeros(self.game.action_size)
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count
        action_probs /= np.sum(action_probs)
        return action_probs


In [ ]:

def draw_board(screen, board):
    # Draw outer border
    pygame.draw.rect(screen, BLACK, (0, 0, BOARD_SIZE, BOARD_SIZE + INPUT_HEIGHT), 2)

    for row in range(8):
        for col in range(8):
            color = WHITE if (row + col) % 2 == 0 else BLACK
            pygame.draw.rect(screen, color, (col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
            piece = board.piece_at(chess.square(col, 7 - row))
            if piece is not None:
                piece_color = "white" if piece.color == chess.WHITE else "black"
                piece_type = get_piece_type(piece)
                piece_image = pygame.image.load(
                    f'C:/Users/faisa/Documents/Games_Section/chess/chess_asstes/{piece_color}_{piece_type}.png'
                )
                piece_image = pygame.transform.scale(piece_image, (SQUARE_SIZE, SQUARE_SIZE))
                screen.blit(piece_image, (col * SQUARE_SIZE, row * SQUARE_SIZE))

    # Draw semi-transparent vertical numbers
    for i in range(8):
        font = pygame.font.Font(None, 36)
        text = font.render(str(8 - i), True, BLACK)
        text.set_alpha(128)  # Set opacity to 128 (0.5)
        text_rect = text.get_rect()
        text_rect.center = (SQUARE_SIZE // 2, i * SQUARE_SIZE + SQUARE_SIZE // 2)
        screen.blit(text, text_rect)

    # Draw semi-transparent horizontal letters
    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    for i in range(8):
        font = pygame.font.Font(None, 36)
        text = font.render(letters[i], True, BLACK)
        text.set_alpha(128)  # Set opacity to 128 (0.5)
        text_rect = text.get_rect()
        text_rect.center = (i * SQUARE_SIZE + SQUARE_SIZE // 2, BOARD_SIZE - SQUARE_SIZE // 2)
        screen.blit(text, text_rect)

# Function to reset the game
def reset_game():
    return chess.Board()

def main():
    pygame.init()
    screen = pygame.display.set_mode((BOARD_SIZE, BOARD_SIZE))
    pygame.display.set_caption("Chess Game")

    reason=""
    moves = []  # List to store moves made in the current game
    board = chess.Board()

    clock = pygame.time.Clock()
    game_over = False  # Track whether the game is over
    game_reset = False  # Track whether the game has been reset
    input_text =''
    while not game_over or game_reset:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                return
            elif event.type == KEYDOWN:
                if event.key == K_RETURN:
                    try:
                        move = chess.Move.from_uci(input_text)
                        if move.uci() in generate_moves(board):
                            board.push(move)
                            input_text = ""
                            evaluation = evaluate_board(board)
                            print(f"Evaluation after move: {evaluation}")
                        else:
                            print("Invalid move. Try again.")
                    except ValueError:
                        print("Invalid input. Enter moves in the format 'e2 e4'.")
                    continue
                elif event.key == K_BACKSPACE:
                    input_text = input_text[:-1]

                elif (event.key == K_SPACE):
                    if game_over:
                    # Reset the game when the space key is pressed
                        board = reset_game()
                        game_over = False
                        game_reset = True
                else:
                    input_text += event.unicode

        draw_board(screen, board)
        pygame.display.flip()
        clock.tick(60)


        if not board.is_game_over():
            if not board.turn:
               ai_turn_mcts(board, max_iterations=100, player=1)
            else:
                ai_turn_mcts(board, max_iterations=100, player=2)
        else:
            game_over = True  # Set game_over to True when the game ends
            if board.is_checkmate():
                if board.turn:
                    print( "Player 2 (Black) wins by checkmate!")
                else:
                    print("Player 1 (White) wins by checkmate!")
            elif board.is_stalemate():
                print("The game is a draw due to stalemate.")
            elif board.is_insufficient_material():
                print("The game is a draw due to insufficient material.")
            elif board.is_seventyfive_moves():
                print("The game is a draw due to the seventy-five moves rule.")
            elif board.is_fivefold_repetition():
                print("The game is a draw due to fivefold repetition.")
            else:
                print("The game ended in a draw for an unknown reason.")

        game_reset = False

    pygame.quit()

if __name__ == "__main__":
    main()